In [ ]:
import pandas as pd
import numpy as np
import openpyxl

temp = pd.read_excel('data/template_구인구직.xlsx', engine='openpyxl')

In [ ]:
# function화
def pre_df(df, temp):
    df.fillna( method='pad', inplace=True)
    df.columns = temp.columns
    df['년/월'] = df['년/월'].str.replace('년 ', '', regex=True).replace('월', '', regex=True)
    df['산업_대분류'] = df['산업_대분류'].str.replace('10차_', '', regex=True)
    df['직종_소분류'] = df['직종_소분류'].str.replace('2018직종_', '', regex=True)
    
    return df


def merge_file(year):
    df = pd.read_excel(f'data/구인구직취업현황(월)_{year}.xlsx', engine='openpyxl')
    temp = pd.read_excel('data/template_구인구직.xlsx', engine='openpyxl')

    import warnings

    warnings.filterwarnings('ignore')

    df = pre_df(df, temp)

    temp['년/월'] = temp['년/월'].astype('str')
    temp['구인인원'].fillna(0, inplace=True)
    temp['구직건수'].fillna(0, inplace=True)
    temp['취업건수'].fillna(0, inplace=True)

    temp['구인인원'] = temp['구인인원'].astype('int64')
    temp['구직건수'] = temp['구직건수'].astype('int64')
    temp['취업건수'] = temp['취업건수'].astype('int64')

    temp_year = temp.loc[temp['년/월'].str.startswith(f'{year}')]

    merged = pd.merge(temp_year, df, how='left', on=['년/월', '시도', '산업_대분류', '직종_소분류'])

    merged.drop(columns=['구인인원_x', '구직건수_x', '취업건수_x'], inplace=True)

    merged.columns = temp.columns

    merged.fillna(0, inplace=True)

    # merged.to_csv(f'data/구인구직취업현황_{year}_merged.csv', index=False)
    
    return merged

In [ ]:
df_2018 = merge_file(2018)
df_2019 = merge_file(2019)
df_2020 = merge_file(2020)
df_2021 = merge_file(2021)
df_2022 = merge_file(2022)

In [ ]:
dfs = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022], axis=0, ignore_index=True)
dfs.to_csv('data/구인구직취업현황_merged.csv', index=False)

In [ ]:
print(len(temp), len(dfs))

In [ ]:
dfs

In [ ]:
dfs.drop(columns=['직종_소분류']).groupby(['년/월', '시도', '산업_대분류'], sort=False).sum().reset_index()